## **META:**
- Criar um sistema com cadastro de usuários e nível de acesso de cada um;
- Treinar um modelo de ML que me informará se devo ou não dar um empréstimo à ele;
- BONÚS: Dizer até qual valor seria seguro emprestar

## **TRADUÇÃO DAS VARIAVEIS**
- Person Income = Rendimento Anual
- Age = Idade
- person_home_ownership	= Propriedade de casa própria
- person_emp_length = Tempo no emprego (em anos)
- loan_intent	= Intenção de Empréstimo
- loan_grade = Grau do Empréstimo
- loan_amnt	= Montante do Empréstimo
- loan_int_rate	= Taxa de Juros
- loan_status = Status do empréstimo (0 não é inadimplente, 1 é inadimplente)
- loan_percent_income = Renda percentual
- cb_person_default_on_file	= Inadimplência histórica
- cb_preson_cred_hist_length = Comprimento do histórico de crédito




In [78]:
## IMPORTANDO BIBLIOTECAS
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [79]:
# Lendo o arquivo CSV
df = pd.read_csv("/home/iago/Documentos/desafio-01/credit_risk_dataset.csv")

In [80]:
# Verificando os 10 primeiros dados
df.head(10)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
5,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2
6,26,77100,RENT,8.0,EDUCATION,B,35000,12.42,1,0.45,N,3
7,24,78956,RENT,5.0,MEDICAL,B,35000,11.11,1,0.44,N,4
8,24,83000,RENT,8.0,PERSONAL,A,35000,8.90,1,0.42,N,2
9,21,10000,OWN,6.0,VENTURE,D,1600,14.74,1,0.16,N,3


In [81]:
# Verificando os 10 últimos dados
df.tail(10)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
32571,60,45600,RENT,1.0,VENTURE,B,20000,10.00,1,0.44,N,26
32572,52,52000,OWN,0.0,PERSONAL,A,9600,8.49,0,0.18,N,22
32573,56,90000,MORTGAGE,0.0,PERSONAL,A,7200,6.17,0,0.08,N,19
32574,52,65004,RENT,4.0,PERSONAL,D,20000,15.58,1,0.31,Y,19
32575,52,64500,RENT,0.0,EDUCATION,B,5000,11.26,0,0.08,N,20
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,N,30
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,N,19
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,N,28
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,N,26
32580,66,42000,RENT,2.0,MEDICAL,B,6475,9.99,0,0.15,N,30


In [82]:
# Verificando o tipo de cada coluna
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


In [88]:
# Normalizando os valores que estão como object
new_person_home = LabelEncoder()
new_loan_intent = LabelEncoder()
new_loan_grade = LabelEncoder()
new_cb_person_default_on_file = LabelEncoder()
df['person_home_ownership'] = new_person_home.fit_transform(df['person_home_ownership'])
df['loan_intent'] = new_loan_intent.fit_transform(df['loan_intent'])
df['loan_grade'] = new_loan_grade.fit_transform(df['loan_grade'])
df['cb_person_default_on_file'] = new_cb_person_default_on_file.fit_transform(df['cb_person_default_on_file'])
df.head(10)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,3,123.0,4,3,35000,16.02,1,0.59,1,3
1,21,9600,2,5.0,1,1,1000,11.14,0,0.10,0,2
2,25,9600,0,1.0,3,2,5500,12.87,1,0.57,0,3
3,23,65500,3,4.0,3,2,35000,15.23,1,0.53,0,2
4,24,54400,3,8.0,3,2,35000,14.27,1,0.55,1,4
5,21,9900,2,2.0,5,0,2500,7.14,1,0.25,0,2
6,26,77100,3,8.0,1,1,35000,12.42,1,0.45,0,3
7,24,78956,3,5.0,3,1,35000,11.11,1,0.44,0,4
8,24,83000,3,8.0,4,0,35000,8.90,1,0.42,0,2
9,21,10000,2,6.0,5,3,1600,14.74,1,0.16,0,3


In [91]:
df.corr()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
person_age,1.000000,0.173202,-0.032506,0.163106,0.035518,0.014218,0.050787,0.012580,-0.021629,-0.042411,0.005807,0.859133
person_income,0.173202,1.000000,-0.203177,0.134268,0.001527,-0.001022,0.266820,0.000792,-0.144449,-0.254471,-0.003613,0.117987
person_home_ownership,-0.032506,-0.203177,1.000000,-0.231736,0.011004,0.120438,-0.130776,0.140454,0.211600,0.142129,0.063004,-0.025534
person_emp_length,0.163106,0.134268,-0.231736,1.000000,0.021749,-0.047276,0.113082,-0.056405,-0.082489,-0.054111,-0.027728,0.144699
loan_intent,0.035518,0.001527,0.011004,0.021749,1.000000,-0.008042,-0.004597,-0.001357,-0.065575,0.000986,-0.003950,0.035251
loan_grade,0.014218,-0.001022,0.120438,-0.047276,-0.008042,1.000000,0.145799,0.933684,0.373080,0.123021,0.537054,0.015069
loan_amnt,0.050787,0.266820,-0.130776,0.113082,-0.004597,0.145799,1.000000,0.146813,0.105376,0.572612,0.039081,0.041967
loan_int_rate,0.012580,0.000792,0.140454,-0.056405,-0.001357,0.933684,0.146813,1.000000,0.335133,0.120314,0.501072,0.016696
loan_status,-0.021629,-0.144449,0.211600,-0.082489,-0.065575,0.373080,0.105376,0.335133,1.000000,0.379366,0.179141,-0.015529
loan_percent_income,-0.042411,-0.254471,0.142129,-0.054111,0.000986,0.123021,0.572612,0.120314,0.379366,1.000000,0.035910,-0.031690
